In [2]:
# 모든 .txt 파일 삭제 (IPython Notebook 셀)
from pathlib import Path
import os

# 대상 디렉토리 경로
TARGET_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data")

count = 0
for txt_file in TARGET_DIR.glob("*.txt"):
    try:
        txt_file.unlink()  # 파일 삭제
        count += 1
    except Exception as e:
        print(f"삭제 실패: {txt_file} ({e})")

print(f"[RESULT] 총 {count}개의 .txt 파일을 삭제 완료.")


[RESULT] 총 2586개의 .txt 파일을 삭제 완료.


In [4]:
# === YOLOv8 자동 라벨링 (클래스별 서로 다른 confidence 적용 + 지정 순서로 저장) ===
from ultralytics import YOLO
from pathlib import Path
import cv2
import math

# ====== 경로 설정 ======
WEIGHT = Path("/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250904_data_study_1/weights/best.pt")
IMG_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data")
IMG_EXTS = [".jpg", ".jpeg", ".png"]

# ====== 원하는 "저장용 클래스 순서" (요청하신 순서) ======
desired_order = [
    "Divot",
    "Fixed_Divot",
    "Dieased_Grass",  # 사용자가 제공한 철자 그대로 둠
    "Pole",
    "Obstacle",
    "Sprinkler",
    "Drain",
    "Golf ball",
]

# ====== 클래스별 confidence threshold (원하면 자유롭게 조정) ======
# 키는 "클래스 이름" (위 desired_order 기준), 값은 임계값(0~1)
per_class_conf = {
    "Divot":        0.75,
    "Fixed_Divot":  0.75,
    "Dieased_Grass":0.75,
    "Pole":         0.75,
    "Obstacle":     0.75,
    "Sprinkler":    0.75,
    "Drain":        0.75,
    "Golf ball":    0.75,
}
DEFAULT_CONF = 0.75  # 사전에 없는 클래스 이름이 들어올 경우 기본값

# ====== 모델 로드 ======
model = YOLO(str(WEIGHT))
model_names = model.names  # ex) {0: 'Divot', 1: 'Fixed_Divot', ...} 또는 list
if isinstance(model_names, dict):
    idx2name = {int(k): v for k, v in model_names.items()}
else:
    idx2name = {i: n for i, n in enumerate(model_names)}

# ====== "모델 클래스 인덱스" -> "저장용 클래스 인덱스" 매핑 만들기 ======
# 1) 이름이 정확히 매칭되면 그 이름의 위치(desired_order index)로 매핑
# 2) 이름이 매칭 안 되면 경고를 남기고 "동일 인덱스"로 저장(최소한 깨지지 않게)
name2desiredidx = {n: i for i, n in enumerate(desired_order)}
modelidx_to_saveidx = {}
unmatched = []

for midx, mname in idx2name.items():
    if mname in name2desiredidx:
        modelidx_to_saveidx[midx] = name2desiredidx[mname]
    else:
        unmatched.append((midx, mname))
        modelidx_to_saveidx[midx] = midx  # fallback: 인덱스 그대로

if unmatched:
    print("[WARN] 아래 모델 클래스 이름은 desired_order와 매칭되지 않아 '동일 인덱스'로 저장합니다:")
    for midx, mname in unmatched:
        print(f"  - model idx {midx}: '{mname}'")

# ====== 이미지 수집 ======
img_files = [p for p in IMG_DIR.iterdir() if p.suffix.lower() in IMG_EXTS]
print(f"[INFO] Found {len(img_files)} images")

saved_box_count = 0
saved_per_class = {i: 0 for i in range(len(desired_order))}

for img_path in img_files:
    # 예측: 여기서는 conf를 낮게(예: 0.01) 주고, 필터링은 아래에서 클래스별로 수행
    results = model.predict(source=str(img_path), conf=0.01, save=False, verbose=False)

    boxes = results[0].boxes
    if boxes is None or len(boxes) == 0:
        continue

    img = cv2.imread(str(img_path))
    if img is None:
        print(f"[WARN] 이미지 열기 실패: {img_path}")
        continue
    h, w = img.shape[:2]

    # 라벨 파일 열기(필요할 때만 생성)
    lines = []
    for b in boxes:
        cls_idx = int(b.cls[0].item())
        conf = float(b.conf[0].item())

        # 모델 클래스 이름을 가져와 per-class threshold 선택
        cls_name = idx2name.get(cls_idx, None)
        # 저장용 클래스 인덱스(요청 순서 기준)
        save_cls_idx = modelidx_to_saveidx.get(cls_idx, cls_idx)

        # 클래스 이름 기반 임계값(없으면 기본값)
        thr = per_class_conf.get(cls_name, DEFAULT_CONF)

        if conf < thr:
            continue

        # xyxy -> xywh (normalized)
        x1, y1, x2, y2 = b.xyxy[0]
        cx = ((x1 + x2) / 2) / w
        cy = ((y1 + y2) / 2) / h
        bw = (x2 - x1) / w
        bh = (y2 - y1) / h

        # 클램프(숫자 오차 대비)
        def clamp(v): 
            return max(0.0, min(1.0, float(v)))
        cx, cy, bw, bh = map(clamp, (cx, cy, bw, bh))

        # 라벨 라인(요청: "저장용 클래스 순서"의 인덱스로 기록)
        lines.append(f"{save_cls_idx} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

        if 0 <= save_cls_idx < len(desired_order):
            saved_per_class[save_cls_idx] += 1
        saved_box_count += 1

    if lines:
        with open(img_path.with_suffix(".txt"), "w") as f:
            f.write("\n".join(lines))

# ====== 리포트 ======
print("====================================")
print(f"[RESULT] 총 이미지 수         : {len(img_files)}")
print(f"[RESULT] 저장된 라벨(박스) 수 : {saved_box_count}")
print("[RESULT] 클래스별 저장된 박스 수 (저장용 순서 기준):")
for i, name in enumerate(desired_order):
    print(f"  - {i:02d} {name:15s}: {saved_per_class.get(i,0)}")
print("====================================")
print("[NOTE] per-class threshold:", per_class_conf)


[INFO] Found 7794 images
[RESULT] 총 이미지 수         : 7794
[RESULT] 저장된 라벨(박스) 수 : 5180
[RESULT] 클래스별 저장된 박스 수 (저장용 순서 기준):
  - 00 Divot          : 793
  - 01 Fixed_Divot    : 3528
  - 02 Dieased_Grass  : 585
  - 03 Pole           : 74
  - 04 Obstacle       : 0
  - 05 Sprinkler      : 1
  - 06 Drain          : 72
  - 07 Golf ball      : 127
[NOTE] per-class threshold: {'Divot': 0.75, 'Fixed_Divot': 0.75, 'Dieased_Grass': 0.75, 'Pole': 0.75, 'Obstacle': 0.75, 'Sprinkler': 0.75, 'Drain': 0.75, 'Golf ball': 0.75}
